In [ ]:
import sys
sys.path.append('../..')
import pandas as pd
import pandas.arrays as pdarray
from src.cleaning import integrity_checks
import src.cleaning.schema_anon_data as anon_data_schema
import numpy as np
import src.cleaning as clean
import matplotlib.pyplot as plt

In [ ]:
sales_data = pd.read_csv('../../data/anon_data.csv')

In [ ]:
sales_data.dtypes

In [ ]:
# Can columns be converted to appropriate data types?
file_schema = {
  'CustomerId': int,
  'SalesOrderNumber': int,
  'SalesOrderLineNumber': int,
  'Qty': int,
  'Brand': 'string',
  'ProductType': 'string',
  'ProductSupplierID': int,
  'Revenue': float,
  'Profit': float,
  'Freight cost': float,
  'SalesTeamName': 'string',
}
failing_columns = list()
for column_name, column_type in file_schema.items():
    try:
        if column_type in [int, float]:
            pd.to_numeric(sales_data[column_name])
        sales_data[column_name].astype(column_type)
    except Exception as e:
        failing_columns.append(column_name)
for i in failing_columns:
    print(i)

# SalesOrderNumber is due to preceeding O instead of 0
# Qty is because of commas in numbers e.g. 1,000
# ProductSupplierId is because of Null values

# Data requires cleaningand formatting before proceeding on further investigation

In [ ]:
sales_data.isna().sum()
# Nulls in:
# Brand: Problem as don't know the item being sold
# Product Type: Problem as don't know the item being sold
# Product Supplier Id: Not a problem is some stuff is refurbished or refunded and then resold?
                        # Question to ask

In [ ]:
# Are null brands the same rows as null product type?
brand_nulls = sales_data.loc[sales_data['Brand'].isna(), :].index
product_type_nulls = sales_data.loc[sales_data['ProductType'].isna(), :].index
(brand_nulls == product_type_nulls).all()
# Yes so removing one will also solve the other

In [ ]:
# Data type enforced data
data_schema = anon_data_schema.schema
typed_data = data_schema.enforce_schema(dataframe=sales_data)

In [ ]:
typed_data.dtypes

In [ ]:
# Check uniqueness before and after
sales_data.nunique()


In [ ]:
typed_data.nunique()
# Qty uniqueness has decreased as numbers that contained commas are now actual numbers so more likely to exist already, e.g. 1000
# SalesOrderNumber uniqueness decreased as some floats are read in as strings and some as number, example give below. So when standardised, uniqueness decreases

In [ ]:
# Describe the data
typed_data.describe()

In [ ]:
# Each sales order number should be a sale to only one customer, sales order number should not cover multiple customers
integrity_checks.mapping_check(typed_data, groupby_column='sales_order_number', check_column='customer_id', number_unique_values=1)
# Results agree

In [ ]:
# Should each customer only belong to one sales team?
integrity_checks.mapping_check(typed_data, groupby_column='customer_id', check_column='sales_team_name', number_unique_values=1)
# Results seem to suggest no
# Something to investigate if some have multiple sales teams like Corporate and SMB

In [ ]:
# Are all items in an order shipped on the same date?
integrity_checks.mapping_check(typed_data, groupby_column='sales_order_number', check_column='ship_date', number_unique_values=1)
# Apparently not, something to investigate, can we save money by shipping all together rather than separate?
# Questions: Are the origin points the same, do they have the same product supplier? Do they come straight from product supplier or our warehouse?

In [ ]:
# Do Can the same SalesOrderLineNumber appear twice on a sales order?
integrity_checks.mapping_check(typed_data, groupby_column=['sales_order_number', 'sales_order_line_number'], check_column='ship_date', number_unique_values=1)
# Some have same item on same line shipped on different days, suggests that some of the items are not available so have to be shipped later
# Althoug for 496125, all come from same supplier, third one has later date, lower revenue, but higher profit, how???? Different webcam?

In [ ]:
# Any 0 quantity?
typed_data.loc[typed_data['quantity'] == 0, :]
# No

In [ ]:
# Any quantity < 0?
typed_data.loc[typed_data['quantity'] < 0, :]
# Yes, returns?


In [ ]:
# Is revenue always positive when quantity is positive?
typed_data.loc[((typed_data['quantity'] >  0) & (typed_data['revenue'] < 0)), :]
# 2 Rows, likely a mistake 

In [ ]:
# Is profit always positive when revenue is positive?
typed_data.loc[((typed_data['revenue'] >  0) & (typed_data['profit'] < 0)), :]
# No, 9606 rows, possibly because of cost of shipping? Or promotional offers? or advertising?

In [ ]:
# Is shipping cost ever negative?
typed_data.loc[typed_data['freight_cost'] < 0, :]
# Yes? Why?
# Can we ship stuff and the product supplier pay us to ship it?

In [ ]:

within_order_shipping_dates = typed_data.groupby('sales_order_number')['ship_date'].unique()
def days_between(datetimes: pdarray.DatetimeArray):
    days_between = list()
    if len(datetimes) == 1:
        return 0
    else:
        time_order = datetimes.argsort()
        for i in range(len(time_order)-1):
            days_between.append((datetimes[time_order[i+1]]-datetimes[time_order[i]]).days)
        return np.mean(days_between)
    
days_between_shipments = within_order_shipping_dates.apply(days_between).sort_values(ascending=False)
days_between_shipments

In [ ]:
# Where do the nulls come from?
sales_team_grouped = typed_data.groupby('sales_team_name').count()
brand_nulls = typed_data[typed_data['brand'].isna()].groupby('sales_team_name').count()
supplier_nulls = typed_data[typed_data['product_supplier_id'].isna()].groupby('sales_team_name').count()
supplier_nulls

In [ ]:
# When did the brand nulls occur?
brand_nulls = typed_data[typed_data['brand'].isna()]
print(brand_nulls['ship_date'].min())
print(brand_nulls['ship_date'].max())

In [ ]:
cleaned_data = clean.clean_data(sales_data)
cleaned_data

In [ ]:
# Outliers by sales_team in profit
# Data is not normally distributed so using quartiles and IQR
grouped_data = typed_data.groupby('sales_team_name')['profit']
medians = grouped_data.median()
lower_quartile = grouped_data.quantile(0.25)
upper_quartile = grouped_data.quantile(0.75)
iqr = upper_quartile - lower_quartile
upper_outlier = upper_quartile + 1.5*iqr
lower_outlier = lower_quartile - 1.5*iqr
top_percentile = grouped_data.quantile(0.88)
lower_percentile = grouped_data.quantile(0.02)
top_percentile - upper_outlier
lower_percentile - lower_outlier
# Removing outliers on this rule would result in removing approximately 15% of the data = 73306 data points which is unacceptable



In [ ]:
plt.scatter(typed_data['quantity'], typed_data['profit'])
# Looking at the plot, some data could be removed as outliers e.g. profit over 150,000 and quanitity over 8000

In [ ]:
# 3 would be removed for profit 
typed_data.loc[typed_data['profit']>150_000, :]
# 2 would be removed by quantity > 0 and profit > revenue
# One more to be removed die to outlier with huge profit not comparable to revenue

In [ ]:
# Two would be removed for quantity 
typed_data.loc[typed_data['quantity']>8_000, :]
# Keep these are data makes sense, would have to be sold at severe discount though

In [ ]:
plt.scatter(typed_data['quantity'], typed_data['revenue'])

In [ ]:
typed_data.loc[typed_data['revenue']>900_000, :]
# Keep as price makes sense

In [ ]:
grouped_count = typed_data.groupby('customer_id').count()
grouped_count.loc[grouped_count.ship_date > 5000, :]

In [ ]:
# Questionable data, returns of 140 all on the same day, have they been processed in batches of 140? If so, whey different revenue, previously bought at different prices, different models of security camera?
# Maybe client was trying out all camera models and choosing one. No good reason to remove, something to raise
cleaned_data.loc[cleaned_data['customer_id']==100007165, :]

In [ ]:
typed_data.loc[typed_data['customer_id']==100007165, :]

In [ ]:
sales_data.loc[sales_data['CustomerId']==100007165, :]
# argument to remove as there is sales data going back to 2021, and these are not listed